In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_data_dir: Path
    test_data_dir: Path
    transformed_train_data_dir: Path
    transformed_test_data_dir: Path


In [6]:
from RiskScorePrediction.constants import *
from RiskScorePrediction.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            train_data_dir = config.train_data_dir,
            test_data_dir = config.test_data_dir,
            transformed_train_data_dir = config.transformed_train_data_dir,
            transformed_test_data_dir = config.transformed_test_data_dir
        )

        return data_transformation_config

In [8]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from RiskScorePrediction import logger

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

        self.numerical_columns = [
            'Age', 'AnnualIncome', 'CreditScore', 'Experience', 'LoanAmount',
            'LoanDuration', 'NumberOfDependents', 'MonthlyDebtPayments',
            'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
            'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
            'PreviousLoanDefaults', 'PaymentHistory', 'LengthOfCreditHistory',
            'SavingsAccountBalance', 'CheckingAccountBalance', 'TotalAssets',
            'TotalLiabilities', 'MonthlyIncome', 'UtilityBillsPaymentHistory',
            'JobTenure', 'NetWorth', 'BaseInterestRate', 'InterestRate',
            'MonthlyLoanPayment', 'TotalDebtToIncomeRatio', 'LoanApproved'
        ]

        self.categorical_columns = [
            'EmploymentStatus', 'EducationLevel',
            'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose'
        ]

    def get_data_transformation(self):
        try:
            
            num_pipeline = Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ])

            
            cat_pipeline = Pipeline([
                ('imputer', SimpleImputer(strategy="most_frequent")),
                ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
            ])

           
            logger.info(f"Categorical columns: {self.categorical_columns}")
            logger.info(f"Numerical columns: {self.numerical_columns}")

           
            preprocessor = ColumnTransformer([
                ("num_pipeline", num_pipeline, self.numerical_columns),
                ("cat_pipeline", cat_pipeline, self.categorical_columns)
            ])

            return preprocessor

        except Exception as e:
            logger.error(f"Error in get_data_transformation: {str(e)}")
            raise e

    def initiate_data_transformation(self):
        try:
           
            train_df = pd.read_csv(self.config.train_data_dir)
            test_df = pd.read_csv(self.config.test_data_dir)

            logger.info("Successfully loaded train and test datasets.")

            
            preprocessing_obj = self.get_data_transformation()

            target_column_name = "RiskScore"

           
            input_feature_train_df = train_df.drop(columns=[target_column_name], axis=1)
            target_feature_train_df = train_df[target_column_name]

            input_feature_test_df = test_df.drop(columns=[target_column_name], axis=1)
            target_feature_test_df = test_df[target_column_name]

            
            input_feature_train_arr = preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessing_obj.transform(input_feature_test_df)

            
            one_hot_feature_names = preprocessing_obj.named_transformers_["cat_pipeline"] \
                .named_steps["one_hot_encoder"].get_feature_names_out(self.categorical_columns)

            
            final_column_names = self.numerical_columns + list(one_hot_feature_names)

            logger.info(f"Transformed training data shape: {input_feature_train_arr.shape}")
            logger.info(f"Transformed test data shape: {input_feature_test_arr.shape}")

            
            train_arr = pd.DataFrame(
                np.c_[input_feature_train_arr, np.array(target_feature_train_df)],
                columns=final_column_names + [target_column_name]
            )
            test_arr = pd.DataFrame(
                np.c_[input_feature_test_arr, np.array(target_feature_test_df)],
                columns=final_column_names + [target_column_name]
            )

            
            train_arr.to_csv(self.config.transformed_train_data_dir, index=False)
            test_arr.to_csv(self.config.transformed_test_data_dir, index=False)

            logger.info("Data transformation completed successfully.")
            logger.info(f"Size of training data: {train_arr.shape}")
            logger.info(f"Size of test data: {test_arr.shape}")

        except Exception as e:
            logger.error(f"Error in initiate_data_transformation: {str(e)}")
            raise e


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.initiate_data_transformation()

except Exception as e:
    raise e

[2025-01-30 23:08:02,849: INFO: common: yaml file : config\config.yaml loaded  sucessfully]
[2025-01-30 23:08:02,854: INFO: common: yaml file : params.yaml loaded  sucessfully]
[2025-01-30 23:08:02,862: INFO: common: yaml file : schema.yaml loaded  sucessfully]
[2025-01-30 23:08:02,864: INFO: common: created directory at :artifacts]
[2025-01-30 23:08:02,864: INFO: common: created directory at :artifacts/data_transformation]
[2025-01-30 23:08:02,958: INFO: 3377191899: Successfully loaded train and test datasets.]
[2025-01-30 23:08:02,960: INFO: 3377191899: Categorical columns: ['EmploymentStatus', 'EducationLevel', 'MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose']]
[2025-01-30 23:08:02,961: INFO: 3377191899: Numerical columns: ['Age', 'AnnualIncome', 'CreditScore', 'Experience', 'LoanAmount', 'LoanDuration', 'NumberOfDependents', 'MonthlyDebtPayments', 'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory', 'Previ